In [1]:
import pandas as pd
from sklearn.model_selection import cross_val_score
from fuzzywuzzy import fuzz,process

# Primary DataSet:-

Source:- https://www.kaggle.com/PromptCloudHQ/imdb-data/data

In [2]:
db = pd.read_csv('IMDB-Movie-Data.csv')
db.drop('Unnamed: 12',axis=1,inplace=True)

In [3]:
db.head(10)

,Rank,Title,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore
0,1,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074.0,333.13,76.0
1,2,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7.0,485820.0,126.46,65.0
2,3,Split,"Horror,Thriller",Three girls are kidnapped by a man with a diag...,M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",2016,117,7.3,157606.0,138.12,62.0
3,4,Sing,"Animation,Comedy,Family","In a city of humanoid animals, a hustling thea...",Christophe Lourdelet,"Matthew McConaughey,Reese Witherspoon, Seth Ma...",2016,108,7.2,60545.0,270.32,59.0
4,5,Suicide Squad,"Action,Adventure,Fantasy",A secret government agency recruits some of th...,David Ayer,"Will Smith, Jared Leto, Margot Robbie, Viola D...",2016,123,6.2,393727.0,325.02,40.0
5,6,The Great Wall,"Action,Adventure,Fantasy",European mercenaries searching for black powde...,Yimou Zhang,"Matt Damon, Tian Jing, Willem Dafoe, Andy Lau",2016,103,6.1,56036.0,45.13,42.0
6,7,La La Land,"Comedy,Drama,Music",A jazz pianist falls for an aspiring actress i...,Damien Chazelle,"Ryan Gosling, Emma Stone, Rosemarie DeWitt, J....",2016,128,8.3,258682.0,151.06,93.0
7,8,Mindhorn,Comedy,A has-been actor best known for playing the ti...,Sean Foley,"Essie Davis, Andrea Riseborough, Julian Barrat...",2016,89,6.4,2490.0,NaN,71.0
8,9,The Lost City of Z,"Action,Adventure,Biography","A true-life drama, centering on British explor...",James Gray,"Charlie Hunnam, Robert Pattinson, Sienna Mille...",2016,141,7.1,7188.0,8.01,78.0
9,10,Passengers,"Adventure,Drama,Romance",A spacecraft traveling to a distant colony pla...,Morten Tyldum,"Jennifer Lawrence, Chris Pratt, Michael Sheen,...",2016,116,7.0,192177.0,100.01,41.0


# Scraping HTML Pages:-

#### run to get the csv's

In [4]:
from Utility import makeCSVs
makeCSVs()

## Import Data from generated CSVs:-

### Casts:-

In [5]:
cast = pd.read_csv('casts.csv',names=['directorCode','directorName','actor','type','role','awards','pic','notes'],error_bad_lines=False,skiprows=22)
cast.head()

,directorCode,directorName,actor,type,role,awards,pic,notes
0,AA13,T:Pygmalion,Leslie Howard,\Sci,"R:smug professor ""Higgins""",AAN,NaN,NaN
1,AA13,T:Pygmalion,Wendy Hiller,\Inn,"R:flower girl ""Eliza""",AAN,NaN,NaN
2,AA13,T:Pygmalion,Wilfrid Lawson,\Und,"R:friend ""Dolittle""",NaN,NaN,NaN
3,AA13,T:Pygmalion,Scott Sunderland,\Und,"R:betting Colonel ""Pickering""",NaN,NaN,NaN
4,AA13,T:Pygmalion,Marie Lohr,\Und,"R:wife ""Mrs.Higgins""",NaN,NaN,NaN


In [6]:
d = []
for i in cast['directorName']:
    if type(i)==str and i[:2] == 'T:':
        d.append(i[2:])
    else:
        d.append(i)
cast['directorName']= d
cast.head()

,directorCode,directorName,actor,type,role,awards,pic,notes
0,AA13,Pygmalion,Leslie Howard,\Sci,"R:smug professor ""Higgins""",AAN,NaN,NaN
1,AA13,Pygmalion,Wendy Hiller,\Inn,"R:flower girl ""Eliza""",AAN,NaN,NaN
2,AA13,Pygmalion,Wilfrid Lawson,\Und,"R:friend ""Dolittle""",NaN,NaN,NaN
3,AA13,Pygmalion,Scott Sunderland,\Und,"R:betting Colonel ""Pickering""",NaN,NaN,NaN
4,AA13,Pygmalion,Marie Lohr,\Und,"R:wife ""Mrs.Higgins""",NaN,NaN,NaN


### Award:-

In [7]:
awtype = pd.read_csv('awtypes.csv',names=['award','organization','country','colloquial','year','notes'],error_bad_lines=True)
awtype.head()

,award,organization,country,colloquial,year,notes
0,AA,Hollywood Academy of Motion Picture Arts and S...,USA,Oscar,NaN,NaN
1,AAN,Hollywood Academy of Motion Picture Arts and S...,USA,Oscar nomination,NaN,NaN
2,AFI Lifetime,American Film Institute,USA,annual awards since 1973,NaN,NaN
3,AFI77,American Film Institute,USA,Best movies poll 1977,NaN,NaN
4,Baer,Berlinale,F.R.Germany,Berliner Baer(-Gold,Silver,Bronze)


### Actors:-

In [8]:
actor = pd.read_csv('actors.csv',names=['stage','dow','birth','giv','gen','dob','dod','type','orig','pict','notes'],error_bad_lines=True)
actor.head()

,stage,dow,birth,giv,gen,dob,dod,type,orig,pict,notes
0,Willie Aames,NaN,Aames,William,M,1960,199x,RU:,\Am,NaN,NaN
1,Bud Abbott,1939-1956,Abbott,William,M,1895,1974,R:straight,comedian,\Am,NaN
2,Diahnne Abbott,1976-1982,NaN,NaN,F,[1][23],199x,R:sexy,\Am,NaN,Cit(2) Mt(Robert deNiro;)
3,George Abbott,1928-1958,Abbott,George,M,1887,199x,R:playwright,producer,\Am,NaN
4,John Abbott,1937-1975,Abbott,John,M,1905,199x,R:staring eyes,eccentric parts,\Br,NaN


### Main:-

In [9]:
main = pd.read_csv('main.csv',names=['H','title','year','director','prds','prds1','st','prc','prc1','cat','aw','locale','Nt'],error_bad_lines=True)
main.head()

,H,title,year,director,prds,prds1,st,prc,prc1,cat,aw,locale,Nt
0,H1,T:Always Tell Your Wife,1922,D:Se.Hicks,P:Lasky,St:Famous,SD:*,sbw,Dram,aw,lc,CoD(Hitchcock),NaN
1,H2,T:Number Thirteen,1922,D:Hitchcock,P:Hitchcock,St:Islington,SD:Famous,sbw,NaN,NaN,NaN,Nt(unfinished),NaN
2,H3,T:Woman to Woman,1922,D:Hitchcock,P:Balcon,St:B-S-F,SD:Wardour,sbw,Dram,NaN,England,CoD(Cutts) Er(same as GCt27,1926?)
3,H4,T:The Passionate Adventure,1924,D:Hitchcock,P:Balcon,St:Gainsborough,SD:GaumontD,sbw,NaN,NaN,NaN,CoD(Cutts),NaN
4,H5,T:The Blackguard,1925,D:Hitchcock,P:Balcon,St:UFA,SD:Wardour,sbw,NaN,NaN,NaN,CoD(Cutts),NaN


In [10]:
d = []
for i in main['title']:
    if type(i)==str and i[:2] == 'T:':
        d.append(i[2:])
    else:
        d.append(i)
main['title']= d

#######################################

d = []
for i in main['director']:
    if type(i)==str and i[:2] == 'D:':
        d.append(i[2:])
    else:
        d.append(i)
main['director']= d

#######################################

d = []
for i in main['prds']:
    if type(i)==str and i[:2] == 'P:':
        d.append(i[2:])
    else:
        d.append(i)
main['prds']= d

#######################################

d = []
for i in main['prds1']:
    if type(i)==str and i[:3] == 'St:':
        d.append(i[3:])
    else:
        d.append(i)
main['prds1']= d

#######################################

d = []
for i in main['st']:
    if type(i)==str and i[:4] == ' SD:':
        d.append(i[4:])
    else:
        d.append(i)
main['st']= d

In [11]:
main.head()

,H,title,year,director,prds,prds1,st,prc,prc1,cat,aw,locale,Nt
0,H1,Always Tell Your Wife,1922,Se.Hicks,Lasky,Famous,*,sbw,Dram,aw,lc,CoD(Hitchcock),NaN
1,H2,Number Thirteen,1922,Hitchcock,Hitchcock,Islington,Famous,sbw,NaN,NaN,NaN,Nt(unfinished),NaN
2,H3,Woman to Woman,1922,Hitchcock,Balcon,B-S-F,Wardour,sbw,Dram,NaN,England,CoD(Cutts) Er(same as GCt27,1926?)
3,H4,The Passionate Adventure,1924,Hitchcock,Balcon,Gainsborough,GaumontD,sbw,NaN,NaN,NaN,CoD(Cutts),NaN
4,H5,The Blackguard,1925,Hitchcock,Balcon,UFA,Wardour,sbw,NaN,NaN,NaN,CoD(Cutts),NaN


In [12]:
l_name,l_score = [],[]
for i in db['Title']:
    Name = ''
    Score = 0
    if i in main['title']:
        Name=i
        Score=100
    else:
        name,score,x = process.extractOne(i, main['title'])
#        name,score,x = process.extractOne(i, main['title'],scorer=fuzz.token_sort_ratio)
        if score >= 80:
            Name=name
            Score=score
    if Name == '':
        l_name.append('')
        l_score.append(0)
    else:
        l_name.append(Name)
        l_score.append(Score)
db['l_name'],db['l_score'] = l_name,l_score
db.head()

,Rank,Title,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore,l_name,l_score
0,1,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074.0,333.13,76.0,The Blackguard,86
1,2,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7.0,485820.0,126.46,65.0,,0
2,3,Split,"Horror,Thriller",Three girls are kidnapped by a man with a diag...,M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",2016,117,7.3,157606.0,138.12,62.0,It,90
3,4,Sing,"Animation,Comedy,Family","In a city of humanoid animals, a hustling thea...",Christophe Lourdelet,"Matthew McConaughey,Reese Witherspoon, Seth Ma...",2016,108,7.2,60545.0,270.32,59.0,Sing,100
4,5,Suicide Squad,"Action,Adventure,Fantasy",A secret government agency recruits some of th...,David Ayer,"Will Smith, Jared Leto, Margot Robbie, Viola D...",2016,123,6.2,393727.0,325.02,40.0,,0


In [13]:
l_dist = db[['Title','l_name','l_score']]
l_dist.to_csv('lScore.csv')
l_dist.head()

,Title,l_name,l_score
0,Guardians of the Galaxy,The Blackguard,86
1,Prometheus,,0
2,Split,It,90
3,Sing,Sing,100
4,Suicide Squad,,0


In [14]:
award = []
for i in db['l_name']:
    if type(i)==str and len(i)==0:
        award.append('NaN')
    else:
        j = (main['title'] == i).idxmax()
        award.append(main['aw'][j])
db['award'] = award
db.to_csv('merge.csv')
db.head()

,Rank,Title,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore,l_name,l_score,award
0,1,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074.0,333.13,76.0,The Blackguard,86,NaN
1,2,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7.0,485820.0,126.46,65.0,,0,NaN
2,3,Split,"Horror,Thriller",Three girls are kidnapped by a man with a diag...,M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",2016,117,7.3,157606.0,138.12,62.0,It,90,NaN
3,4,Sing,"Animation,Comedy,Family","In a city of humanoid animals, a hustling thea...",Christophe Lourdelet,"Matthew McConaughey,Reese Witherspoon, Seth Ma...",2016,108,7.2,60545.0,270.32,59.0,Sing,100,H-
4,5,Suicide Squad,"Action,Adventure,Fantasy",A secret government agency recruits some of th...,David Ayer,"Will Smith, Jared Leto, Margot Robbie, Viola D...",2016,123,6.2,393727.0,325.02,40.0,,0,NaN


In [15]:
db['Revenue (Millions)'] = db['Revenue (Millions)'].fillna(db['Revenue (Millions)'].median())
db['Metascore'] = db['Metascore'].fillna(db['Metascore'].median())
db['award'] = db['award'].fillna("NaN")
db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 15 columns):
Rank                  1000 non-null int64
Title                 1000 non-null object
Genre                 1000 non-null object
Description           1000 non-null object
Director              1000 non-null object
Actors                1000 non-null object
Year                  1000 non-null object
Runtime (Minutes)     1000 non-null int64
Rating                1000 non-null float64
Votes                 1000 non-null float64
Revenue (Millions)    1000 non-null float64
Metascore             1000 non-null float64
l_name                999 non-null object
l_score               1000 non-null int64
award                 1000 non-null object
dtypes: float64(4), int64(3), object(8)
memory usage: 117.3+ KB


# Encoding:-

In [16]:
db.drop(['Rank','Title','Description','l_name','l_score'],axis=1,inplace=True)
db.head()

,Genre,Director,Actors,Year,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore,award
0,"Action,Adventure,Sci-Fi",James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074.0,333.13,76.0,NaN
1,"Adventure,Mystery,Sci-Fi",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7.0,485820.0,126.46,65.0,NaN
2,"Horror,Thriller",M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",2016,117,7.3,157606.0,138.12,62.0,NaN
3,"Animation,Comedy,Family",Christophe Lourdelet,"Matthew McConaughey,Reese Witherspoon, Seth Ma...",2016,108,7.2,60545.0,270.32,59.0,H-
4,"Action,Adventure,Fantasy",David Ayer,"Will Smith, Jared Leto, Margot Robbie, Viola D...",2016,123,6.2,393727.0,325.02,40.0,NaN


In [17]:
rate = []
for i in db['Rating']:
    if i>=0 and i<2.4:
        rate.append('Terrible')
    elif i>=2.4 and i<=5:
        rate.append('Poor')
    elif i>=5.1 and i<=7.4:
        rate.append('Average')
    else:
        rate.append('Excellent')
db['Rating']=rate

In [18]:
from sklearn.preprocessing import LabelEncoder
ge = LabelEncoder().fit_transform(db['Genre'])
db['Genre'] = ge

ac = LabelEncoder().fit_transform(db['Actors'])
db['Actors'] = ac

di = LabelEncoder().fit_transform(db['Director'])
db['Director'] = di

aw = LabelEncoder().fit_transform(db['award'])
db['award'] = aw


r = LabelEncoder().fit_transform(db['Rating'])
db['Rating'] = r


In [19]:
l = []
for i,j in enumerate(db.Year):
    if j.isnumeric() == False:
        l.append(i)
db.drop(db.index[l],inplace=True)
db.head()

,Genre,Director,Actors,Year,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore,award
0,11,267,183,2014,121,1,757074.0,333.13,76.0,146
1,85,519,737,2012,124,0,485820.0,126.46,65.0,146
2,195,392,417,2016,117,0,157606.0,138.12,62.0,146
3,92,108,659,2016,108,0,60545.0,270.32,59.0,119
4,7,139,971,2016,123,0,393727.0,325.02,40.0,146


# Spliting:-

In [20]:
OTT = db[db.Year=='2016']
other =  db[db.Year!='2016']
X = other.drop(['Rating'],axis=1)
Y = other['Rating']

# Machine Learning:-

In [46]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LassoLars
from sklearn.svm import SVR
import random

Xtrain,Xtest,Ytrain,Ytest = train_test_split( X, Y, test_size=0.33, random_state=random.randint(1,101))

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

In [23]:
l_clf = LassoLars(alpha=.1).fit(Xtrain,Ytrain)
lars = l_clf.predict(Xtest)
s_clf = SVR().fit(Xtrain,Ytrain)
svr = s_clf.predict(Xtest)

import numpy as np
lars=np.round_(lars)
svr=np.round_(svr)

### LassoLars Score:-

In [58]:
print("MAE = %.2f"%mean_absolute_error(lars,Ytest))
print("R2 = %.2f"%r2_score(lars,Ytest))
print("Accuracy = %.2f"%accuracy_score(lars,Ytest))

MAE = 0.31
R2 = 0.00
Accuracy = 0.72


### SVR Score:-

In [34]:
print("SVR:-")
print("MAE = %.2f"%mean_absolute_error(svr,Ytest))
print("R2 = %.2f"%r2_score(svr,Ytest))
print("Accuracy = %.2f"%accuracy_score(svr,Ytest))

SVR:-
MAE = 0.30
R2 = 0.00
Accuracy = 0.72


# Out Of Time (2016) :-

In [28]:
ootX,ootY=OTT.drop(['Rating'],axis=1),OTT['Rating']
oot1 = l_clf.predict(ootX)
oot2 = s_clf.predict(ootX)
oot1=np.round_(oot1)
oot2=np.round_(oot2)

### LassoLars Score:-

In [40]:
print("MAE = %.2f"%mean_absolute_error(oot1,ootY))
print("R2 = %.2f"%r2_score(oot1,ootY))

MAE = 0.34
R2 = 0.00


### SVR Score:-

In [38]:
print("MAE = %.2f"%mean_absolute_error(oot2,ootY))
print("R2 = %.2f"%r2_score(oot2,ootY))

MAE = 0.34
R2 = 0.00


# FUN FUNCTION:-

In [79]:
def fun_name(movie_name):
    fun = pd.read_csv('merge.csv')
    name,score,x = process.extractOne(movie_name, fun['Title'])
    l = (fun['Title'] == name).idxmax()
    if l == 0 and fun['Title'][l]==name:
        return fun['Rating'][l]
    else:
        return "Unknown"

In [80]:
print(fun_name('Guardians of the Gala'))

8.1


In [81]:
print(fun_name('DOOM45'))

Unknown
